In [1]:
import string
import csv
import re
import codecs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import feature_extraction
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
import nltk 

In [2]:
lang=pd.read_csv("D:\HP1\Downloads\dataset.csv")

In [3]:
lang=lang[0:2000]

In [4]:
lang

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch
...,...,...
1995,in johnson joined the architecture department...,English
1996,el trazado que coincide con la actual posición...,Spanish
1997,สวิตเซอร์แลนด์เป็นตัวอย่างหายากของประเทศที่มีก...,Thai
1998,வட அமெரிக்கா ஐரோப்பா மற்றும் ஆத்திரேலியா போன்...,Tamil


In [5]:
lang.isnull().sum()

Text        0
language    0
dtype: int64

In [56]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(Text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(Text)]


lang['text_lemmatized']= lang.Text.apply(lemmatize_text)
print(lang['text_lemmatized'])

0       [klement, gottwaldi, surnukeha, palsameeriti, ...
1       [sebe, joseph, pereira, thomas, på, eng, the, ...
2       [ถนนเจริญกรุง, อักษรโรมัน, thanon, charoen, kr...
3       [விசாகப்பட்டினம், தமிழ்ச்சங்கத்தை, இந்துப், பத...
4       [de, spons, behoort, tot, het, geslacht, halic...
                              ...                        
1995    [in, johnson, joined, the, architecture, depar...
1996    [el, trazado, que, coincide, con, la, actual, ...
1997    [สวิตเซอร์แลนด์เป็นตัวอย่างหายากของประเทศที่มี...
1998    [வட, அமெரிக்கா, ஐரோப்பா, மற்றும், ஆத்திரேலியா,...
1999    [segundo, lívio, a, causa, imediata, desta, gu...
Name: text_lemmatized, Length: 2000, dtype: object


In [6]:
for char in string.punctuation:
    print(char,end=" ")
translate_table=dict((ord(char),None) for char in string.punctuation)    

! " # $ % & ' ( ) * + , - . / : ; < = > ? @ [ \ ] ^ _ ` { | } ~ 

In [1]:
data_lang=[]
for i,line in lang.iterrows():
    line=line['Text']
    if len(line)!=0:
        line=re.sub(r"\d+", "",line)
        line=re.sub(r"[a-zA-Z]+","",line)
        line=line.translate(translate_table)
        data_lang.append(line)            
        
languag=[]        
for i,line in lang.iterrows():
    line=line['language']
    if len(line)!=0:
        languag.append(line)
        

NameError: name 'lang' is not defined

In [8]:
df=pd.DataFrame({"Text":data_lang,"language":languag} )

In [9]:
df

,Text,language
0,ä ä í – ...,Estonian
1,å,Swedish
2,ถนนเจริญกรุง อักษรโรมัน เริ่มตั้งแต่ถนนสนาม...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,,Dutch
...,...,...
1995,...,English
1996,ó áñ í é í ó ...,Spanish
1997,สวิตเซอร์แลนด์เป็นตัวอย่างหายากของประเทศที่มีก...,Thai
1998,வட அமெரிக்கா ஐரோப்பா மற்றும் ஆத்திரேலியா போன்...,Tamil


In [10]:
df.shape

(2000, 2)

In [11]:
X,y=df['Text'],df['language']
y

0        Estonian
1         Swedish
2            Thai
3           Tamil
4           Dutch
          ...    
1995      English
1996      Spanish
1997         Thai
1998        Tamil
1999    Portugese
Name: language, Length: 2000, dtype: object

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1600,)
(400,)
(1600,)
(400,)


In [14]:
vectorizer=feature_extraction.text.TfidfVectorizer(ngram_range=(1,3),analyzer='char')

In [15]:
pipeline_clf=Pipeline([('vectorizer',vectorizer),('clf',linear_model.LogisticRegression())])

In [16]:
pipeline_clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', ngram_range=(1, 3))),
                ('clf', LogisticRegression())])

In [17]:
y_predicted=pipeline_clf.predict(X_test)

In [18]:
acc=(metrics.accuracy_score(y_test,y_predicted))

In [19]:
print(acc)

0.8575


In [20]:
import pickle
lrfile=open('Langmodel.pckl','wb')
pickle.dump(pipeline_clf,lrfile)
lrfile.close()

In [21]:
def lang_detect(text):
    import pickle
    import numpy as np
    import string
    import re
    translate_table=dict((ord(char),None) for char in string.punctuation) 
    
    global LanguagedetectionModel
    Languagedetectionfile=open('Langmodel.pckl','rb')
    LanguagedetectionModel=pickle.load(Languagedetectionfile)
    Languagedetectionfile.close()
    
    text=" ".join(text.split())
    text=text.lower()
    text=re.sub(r"\d+", "",text)
    text=text.translate( translate_table)
    pred=LanguagedetectionModel.predict([text])
    return pred

In [22]:
df['language'].value_counts()

Russian       113
Latin         104
Japanese      102
Tamil          99
Turkish        99
Hindi          98
Portugese      97
Persian        97
Dutch          94
Thai           93
Urdu           92
Swedish        91
Pushto         87
English        86
Spanish        84
Arabic         84
French         83
Indonesian     82
Estonian       82
Korean         81
Chinese        77
Romanian       75
Name: language, dtype: int64

In [23]:
df.drop(df.loc[df['language']=='indonesian'].index,inplace=True)

In [24]:
df['language'].value_counts()

Russian       113
Latin         104
Japanese      102
Tamil          99
Turkish        99
Hindi          98
Portugese      97
Persian        97
Dutch          94
Thai           93
Urdu           92
Swedish        91
Pushto         87
English        86
Spanish        84
Arabic         84
French         83
Indonesian     82
Estonian       82
Korean         81
Chinese        77
Romanian       75
Name: language, dtype: int64

In [23]:
lang_detect("Bonne Année")

array(['French'], dtype=object)

In [24]:
lang_detect("¡Feliz Año Nuevo!")

array(['Latin'], dtype=object)

In [25]:
lang_detect("あけましておめでとう")

array(['Japanese'], dtype=object)

In [26]:
lang_detect("Mutlu yıllar")

array(['Turkish'], dtype=object)

In [27]:
lang_detect("Sťastný nový rok")

array(['Indonesian'], dtype=object)

In [28]:
lang_detect("S novym godom")

array(['Indonesian'], dtype=object)

In [29]:
lang_detect("Feliz Ano Novo")

array(['Indonesian'], dtype=object)

In [30]:
df['language']=='Indonesian'.value_count()

AttributeError: 'str' object has no attribute 'value_count'